In [10]:
#input a vector corresponding to a probability distribution and output an index according to that distribution
function markovrand(nextv)
    if round(sum(nextv), digits = 6) != 1.0
        error(nextv, " is not a stochastic vector")
    end
    maxstates = length(nextv) - 1
    r = rand(Float64)
    current = maxstates
    for i = 0:maxstates
        if r <= nextv[i + 1]
            current = i
            break
        else
            r = r - nextv[i + 1]
        end
    end
    return(current)
end


markovrand (generic function with 1 method)

In [11]:
struct AugMatrix
    mat::Matrix
    init::Vector
end

struct AugDblMatrix
    mat1::Matrix
    mat2::Matrix
    init::Vector
end

In [12]:
#output bass notes in 2*$beats notes
function ghostbassgen(beats::Int, trans::AugMatrix)
    transmat = trans.mat
    iv = trans.init
    nextv = iv
    L = []
    for j = 1:beats
        current = markovrand(nextv)
#print(current, " ")
        bincurrent = digits(current, base = 3, pad = 2)
#print(bincurrent, "\n")
        append!(L, [bincurrent[2], bincurrent[1]])
        nextv = transmat[:, current + 1]
    end
    return(L)
end

ghostbassgen (generic function with 1 method)

In [74]:
#0 = rest, 1 = bass, 2 = LH
                #00    #01  #02   #10  #11  #12  #zeros
rockBassMatrix = [0.1e0 1//6  0   1//6  0.5    0 0 0 0 #00
                    0   1//3  0     0    0    0  0 0 0  #01
                    0   1//6  0.1   1//6  0.5   0 0 0 0  #02
                    0.45 1//3  0.45  1//3 0    0 0 0 0  #10
                    0.45 0    0.45  1//3  0    0 0 0 0  #11
                    0.0  0    0      0   0    0 0 0 0    #12
                    zeros(3,9)]                 #zeros

9×9 Matrix{Float64}:
 0.1   0.166667  0.0   0.166667  0.5  0.0  0.0  0.0  0.0
 0.0   0.333333  0.0   0.0       0.0  0.0  0.0  0.0  0.0
 0.0   0.166667  0.1   0.166667  0.5  0.0  0.0  0.0  0.0
 0.45  0.333333  0.45  0.333333  0.0  0.0  0.0  0.0  0.0
 0.45  0.0       0.45  0.333333  0.0  0.0  0.0  0.0  0.0
 0.0   0.0       0.0   0.0       0.0  0.0  0.0  0.0  0.0
 0.0   0.0       0.0   0.0       0.0  0.0  0.0  0.0  0.0
 0.0   0.0       0.0   0.0       0.0  0.0  0.0  0.0  0.0
 0.0   0.0       0.0   0.0       0.0  0.0  0.0  0.0  0.0

In [75]:
sum(rockBassMatrix, dims=1)

1×9 Matrix{Float64}:
 1.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0

In [76]:
rockBassInitial = [0.1,0, 0.1, 0.4, 0.4, 0, 0, 0, 0]

9-element Vector{Float64}:
 0.1
 0.0
 0.1
 0.4
 0.4
 0.0
 0.0
 0.0
 0.0

In [77]:
rockBass = AugMatrix(rockBassMatrix, rockBassInitial)

AugMatrix([0.1 0.16666666666666666 … 0.0 0.0; 0.0 0.3333333333333333 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.1, 0.0, 0.1, 0.4, 0.4, 0.0, 0.0, 0.0, 0.0])

In [78]:
#test
ghostbassgen(4, rockBass)

8-element Vector{Any}:
 1
 1
 0
 2
 1
 1
 0
 2

In [95]:
rockNoGhostBMatrix = [0.1e0 1//3 1//3 1; 0 1//3 0 0; 0.45e0 1//3 1//3 0; 0.45e0 0 1//3 0]

4×4 Matrix{Float64}:
 0.1   0.333333  0.333333  1.0
 0.0   0.333333  0.0       0.0
 0.45  0.333333  0.333333  0.0
 0.45  0.0       0.333333  0.0

In [105]:
rockNoGhostBInitial = [0.1e0,0, 0,0.5e0,0.4e0, 0, 0, 0, 0]

9-element Vector{Float64}:
 0.1
 0.0
 0.0
 0.5
 0.4
 0.0
 0.0
 0.0
 0.0

In [106]:
rockNoGhostB = AugMatrix([rockNoGhostBMatrix[1:2, 1:2] zeros(2,1) rockNoGhostBMatrix[1:2, 3:4] zeros(2, 4)
    zeros(1, 9)
    rockNoGhostBMatrix[3:4, 1:2] zeros(2,1) rockNoGhostBMatrix[3:4, 3:4] zeros(2, 4)
    zeros(4, 9)
    ],
    rockNoGhostBInitial)

AugMatrix([0.1 0.3333333333333333 … 0.0 0.0; 0.0 0.3333333333333333 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.1, 0.0, 0.0, 0.5, 0.4, 0.0, 0.0, 0.0, 0.0])

In [18]:
function lookup16ride()::Matrix
    r = Matrix(undef, 4, 4)
    r[1,1] = ["!N![^e","[","!N![^e","[","!N![c^e","[","!N![^e","[","!N![^e","[","!N![^e","[","!N![c^e","[","!N![^e","["] #hihat eighths, trash
    #NB Windows midi doesn't have a good way of making a trash hat?  !N! defined as tenuto mark in header.abc
    r[2,1] = ["[^e", "[^e", "[^e", "[^e", "[c^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[c^e", "[^e", "[^e", "[^e"]; #hihat 16
    r[3,1] = ["!>![^e", "!U![^e", "!>![^e", "!U![^e", "!>![c^e", "!U![^e","!>![^e", "!U![^e","!>![^e", "!U![^e", "!>![^e", "!U![^e", "!>![c^e", "!U![^e", "!>![^e", "!U![^e"]; #drive 16
    r[4,1] = ["[^e", "[^e", "[^e", "[^e", "[c^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[c^e", "[^e", "[^e", "[_e"];  #one slide 16
    r[1,2] = ["[^g","[","[^g","[","[c^g","[","[^g","[","[^g","[","[^g","[","[c^g","[","[^g","["] #ride quarter
    r[2,2] = ["[^g", "[^g", "[^g", "[^g", "[c^g", "[^g", "[^g", "[^g", "[^g", "[^g", "[^g", "[^g", "[c^g", "[^g", "[^g", "[^g"]; #ride 16 
    r[3,2] = ["[^g^D", "[^g", "[^g", "[^g", "[c^g^D", "[^g", "[^g", "[^g", "[^g^D", "[^g", "[^g", "[^g", "[c^g^D", "[^g", "[^g", "[^g"]; #ride 16 with step beats
    r[4,2] = ["[^g", "[^g", "[^g^D", "[^g", "[c^g", "[^g", "[^g^D", "[^g", "[^g", "[^g", "[^g^D", "[^g", "[c^g", "[^g", "[^g^D", "[^g" ]; #ride 16 with step ands
    r[1,3] = ["[g","[","[g","[","[cg","[","[g","[","[g","[","[g","[","[cg","[","[g","["] #bell 8
    r[2,3] = ["[g","[g","[","[","[cg","[g","[","[","[g","[g","[","[","[cg","[g","[","["] #bell 16th pairs on beat
    r[3,3] = ["[g","[^g","[g","[^g","[cg","[^g","[g","[^g","[g","[^g","[g","[^g","[cg","[^g","[g","[^g"] #bell drive, offbeat ride
    r[4,3] = ["[","[","[g","[g","[c","[","[g","[g","[","[","[g","[g","[c","[","[g","[g"] #bell pairs on and
    r[1,4] = ["[^D","[","[","[","[c^D","[","[","[","[^D","[","[","[","[c^D","[","[","["]  #step Q
    r[2,4] = ["[^D","[","[^D","[","[c^D","[","[^D","[","[^D","[","[^D","[","[c^D","[","[^D","["] #step 8
    r[3,4] = ["[g^D","[g","[^D","[","[cg^D","[g","[^D","[","[g^D","[g","[^D","[","[cg^D","[g","[^D","["] #step 8 with bell beat pair
    r[4,4] = ["[^D","[","[g^D","[g","[c^D","[","[g^D","[g","[^D","[","[g^D","[g","[c^D","[","[g^D","[g"] #step 8 with bell and pair
    return(r)
end


lookup16ride (generic function with 1 method)

In [19]:
#input a sixteenth note bass rhythm and an 16th note ride pattern, output abc notation
function abctime16patinst16ghost(L, pattern, instr)
    n = length(L)
    r = lookup16ride()
    c = ["]/2", "F]/2", "!Q!c]/2"] #nothing, kick, ghost snare
    m = [c[L[i] + 1] for i = 1:n]
    s = ""
    for i = 1:n
        s = s * r[pattern, instr][(i - 1) % 16 + 1] * m[i]
        if i % 4 == 0
            s = s * " "
        end
        if i % 16 == 0
            s = s * "|\n"
        end
    end
    return(s)
end


abctime16patinst16ghost (generic function with 1 method)

In [20]:
#test
abctime16patinst16ghost(ghostbassgen(8, rockBass), 1, 1)

"!N![^eF]/2[!Q!c]/2!N![^eF]/2[]/2 !N![c^eF]/2[]/2!N![^e]/2[]/2 !N![^eF]/2[F]/2!N![^e]/2[!Q!c]/2 !N![c^eF]/2[F]/2!N![^e]/2[]/2 |\n"

In [21]:
#output linear fill in $beats number of sixteenth notes
function fill16gen(
  beats::Int,
  trans::AugDblMatrix)
    nextv = trans.init
    L = []
    for j = 1:beats
        current = markovrand(nextv)
        append!(L,current)
        if j % 2 == 1
            nextv = trans.mat1[1:6, current + 1]
        else
            nextv = trans.mat2[1:6, current + 1]
        end
    end
    return(L)
end


fill16gen (generic function with 1 method)

In [22]:
rockFillLeftToRight = [
    0.2e0 0.1e0 0.2e0 0 0.3e0 0.2e0; 
    0.4e0 0.2e0 0 0.3e0 0.3e0 0.2e0; 
    0.1e0 0.4e0 0.2e0 0.1e0 0.1e0 0.2e0; 
    0.1e0 0.1e0 0.4e0 0.3e0 0.1e0 0.2e0; 
    0.1e0 0.1e0 0.2e0 0.2e0 0.1e0 0.2e0; 
    0.1e0 0.1e0 0 0.1e0 0.1e0 0]

6×6 Matrix{Float64}:
 0.2  0.1  0.2  0.0  0.3  0.2
 0.4  0.2  0.0  0.3  0.3  0.2
 0.1  0.4  0.2  0.1  0.1  0.2
 0.1  0.1  0.4  0.3  0.1  0.2
 0.1  0.1  0.2  0.2  0.1  0.2
 0.1  0.1  0.0  0.1  0.1  0.0

In [23]:
rockFillRightToLeft = [
    0.4e0 0.2e0 0.1e0 0.1e0 0.3e0 0
    0.2e0 0.4e0 0.2e0 0.2e0 0.2e0 0
    0.1e0 0 0.4e0 0.1e0 0.1e0 0
    0 0.1e0 0.1e0 0.4e0 0.1e0 0
    0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0
    0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.10e1]


6×6 Matrix{Float64}:
 0.4  0.2  0.1  0.1  0.3  0.0
 0.2  0.4  0.2  0.2  0.2  0.0
 0.1  0.0  0.4  0.1  0.1  0.0
 0.0  0.1  0.1  0.4  0.1  0.0
 0.2  0.1  0.1  0.1  0.2  0.0
 0.1  0.2  0.1  0.1  0.1  1.0

In [24]:
rockFillInitial = [0.4e0,0.4e0,0.1e0,0.1e0,0,0] #snare, high, mid, low, kick, rest

6-element Vector{Float64}:
 0.4
 0.4
 0.1
 0.1
 0.0
 0.0

In [25]:
rockFill= AugDblMatrix(rockFillLeftToRight, rockFillRightToLeft, rockFillInitial)

AugDblMatrix([0.2 0.1 … 0.3 0.2; 0.4 0.2 … 0.3 0.2; … ; 0.1 0.1 … 0.1 0.2; 0.1 0.1 … 0.1 0.0], [0.4 0.2 … 0.3 0.0; 0.2 0.4 … 0.2 0.0; … ; 0.2 0.1 … 0.2 0.0; 0.1 0.2 … 0.1 1.0], [0.4, 0.4, 0.1, 0.1, 0.0, 0.0])

In [26]:
#input a sixteenth note fill and a crescendo/descr indicator, output abc notation
function abcfill16(L, crdc)
    n = length(L)
    c = ["c/2","e/2","A/2","G/2","F/2","z/2"]
    m = [c[L[i] + 1] for i = 1:n]
    s = ""
    for i = 1:n
        s = s * m[i]
        if i % 4 == 0
            s = s * " "
        end
        if i % 16 == 12
            if crdc < 0
                s = s * "!>(!"
            elseif 0 < crdc
                s = s * "!<(!"
            end
        end
        if i % 16 == 14
            if crdc < 0
                s = s * "!>)!"
            elseif 0 < crdc
                s = s * "!<)!"
            end
        end
    end
    return(s)
end


abcfill16 (generic function with 1 method)

In [27]:
#test
abcfill16(fill16gen(16, rockFill), 0)

"e/2A/2c/2A/2 e/2G/2c/2e/2 e/2A/2A/2A/2 c/2z/2z/2F/2 "

In [28]:
dynMatrix = [
    0 0 0.2e0 0.2e0 0.1e0 0.1e0 0 0 0 0 0 0; 
    0 0 0.2e0 0.1e0 0.1e0 0.1e0 0 0 0 0 0 0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.2e0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.2e0 0.1e0; 
    0 0 0.1e0 0.2e0 0 0 0.2e0 0.1e0 0 0 0.1e0 0.2e0; 
    0 0 0.2e0 0.2e0 0 0 0.2e0 0.1e0 0 0 0.2e0 0.1e0; 
    0.1e0 0.2e0 0 0 0.1e0 0.2e0 0 0 0.2e0 0.2e0 0 0; 
    0.2e0 0.1e0 0 0 0.1e0 0.2e0 0 0 0.2e0 0.1e0 0 0; 
    0.1e0 0.2e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0;
    0.2e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0; 
    0 0 0 0 0 0 0.1e0 0.1e0 0.1e0 0.2e0 0 0; 
    0 0 0 0 0 0 0.1e0 0.1e0 0.2e0 0.2e0 0 0]


12×12 Matrix{Float64}:
 0.0  0.0  0.2  0.2  0.1  0.1  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.2  0.1  0.1  0.1  0.0  0.0  0.0  0.0  0.0  0.0
 0.2  0.2  0.0  0.0  0.2  0.1  0.1  0.1  0.1  0.2  0.1  0.2
 0.2  0.2  0.0  0.0  0.2  0.1  0.1  0.1  0.2  0.1  0.2  0.1
 0.0  0.0  0.1  0.2  0.0  0.0  0.2  0.1  0.0  0.0  0.1  0.2
 0.0  0.0  0.2  0.2  0.0  0.0  0.2  0.1  0.0  0.0  0.2  0.1
 0.1  0.2  0.0  0.0  0.1  0.2  0.0  0.0  0.2  0.2  0.0  0.0
 0.2  0.1  0.0  0.0  0.1  0.2  0.0  0.0  0.2  0.1  0.0  0.0
 0.1  0.2  0.1  0.2  0.1  0.1  0.1  0.2  0.0  0.0  0.2  0.2
 0.2  0.1  0.2  0.1  0.1  0.1  0.1  0.2  0.0  0.0  0.2  0.2
 0.0  0.0  0.0  0.0  0.0  0.0  0.1  0.1  0.1  0.2  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.1  0.1  0.2  0.2  0.0  0.0

In [29]:
dynFinal = [
    0.1e0 0.2e0 0.2e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0 0; 
    0.2e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0 0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0.2e0 0.1e0 0.2e0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0.1e0; 
    0 0 0 0 0 0 0 0 0 0 0 0; 
    0 0 0 0 0 0 0 0 0 0 0 0; 
    0 0 0 0 0 0 0 0 0 0 0 0; 
    0 0 0 0 0 0 0 0 0 0 0 0; 
    0.1e0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0; 
    0.2e0 0.1e0 0.2e0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0; 
    0 0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.2e0; 
    0 0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0.2e0 0.2e0 0.1e0]


12×12 Matrix{Float64}:
 0.1  0.2  0.2  0.2  0.1  0.1  0.1  0.1  0.1  0.2  0.0  0.0
 0.2  0.1  0.2  0.1  0.1  0.1  0.1  0.1  0.2  0.1  0.0  0.0
 0.2  0.2  0.0  0.0  0.2  0.1  0.2  0.1  0.2  0.2  0.1  0.2
 0.2  0.2  0.0  0.0  0.2  0.1  0.2  0.1  0.2  0.1  0.2  0.1
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.1  0.2  0.1  0.2  0.1  0.2  0.1  0.2  0.0  0.0  0.2  0.2
 0.2  0.1  0.2  0.2  0.1  0.2  0.1  0.2  0.0  0.0  0.2  0.2
 0.0  0.0  0.1  0.2  0.1  0.1  0.1  0.1  0.1  0.2  0.1  0.2
 0.0  0.0  0.2  0.1  0.1  0.1  0.1  0.1  0.2  0.2  0.2  0.1

In [30]:
dynInitial = [0.5e-1,0.5e-1,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.5e-1,0.5e-1]
#pp>, pp<, p>, p<, mp>, mp<, mf>, mf<, f>, f<, ff>, ff<

12-element Vector{Float64}:
 0.05
 0.05
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.05
 0.05

In [31]:
dyn = AugDblMatrix(dynMatrix, dynFinal, dynInitial)

AugDblMatrix([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.1 0.2 … 0.0 0.0; 0.2 0.1 … 0.0 0.0; … ; 0.0 0.0 … 0.1 0.2; 0.0 0.0 … 0.2 0.1], [0.05, 0.05, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05])

In [32]:
rockPatMatrix = [
    #8  Q   b   +
    0   0.4 0.4 0.4 #8 
    0.4 0   0.2 0.2   #Q
    0.2 0.2 0   0.4 #b
    0.4 0.4 0.4 0 #+
    ]



4×4 Matrix{Float64}:
 0.0  0.4  0.4  0.4
 0.4  0.0  0.2  0.2
 0.2  0.2  0.0  0.4
 0.4  0.4  0.4  0.0

In [33]:
sum(rockPatMatrix, dims=1)

1×4 Matrix{Float64}:
 1.0  1.0  1.0  1.0

In [34]:
rockPatInitial = [0.4; 0.4; 0.2; 0]

4-element Vector{Float64}:
 0.4
 0.4
 0.2
 0.0

In [35]:
rockPat = AugMatrix(rockPatMatrix, rockPatInitial)

AugMatrix([0.0 0.4 0.4 0.4; 0.4 0.0 0.2 0.2; 0.2 0.2 0.0 0.4; 0.4 0.4 0.4 0.0], [0.4, 0.4, 0.2, 0.0])

In [36]:
rockInstMatrix = [
   #HH  RC  RB  ST 
    0.1 0.4 0.4   0.4   #HH
    0.6 0.1 0.4   0.4   #RC
    0.2 0.4 0.1   0.2   #RB
    0.1 0.1 0.1   0     #ST
]

4×4 Matrix{Float64}:
 0.1  0.4  0.4  0.4
 0.6  0.1  0.4  0.4
 0.2  0.4  0.1  0.2
 0.1  0.1  0.1  0.0

In [37]:
sum(rockInstMatrix, dims=1)

1×4 Matrix{Float64}:
 1.0  1.0  1.0  1.0

In [38]:
rockInstInitial = [0.5; 0.3; 0; 0.2]

4-element Vector{Float64}:
 0.5
 0.3
 0.0
 0.2

In [39]:
rockInst = AugMatrix(rockInstMatrix, rockInstInitial)

AugMatrix([0.1 0.4 0.4 0.4; 0.6 0.1 0.4 0.4; 0.2 0.4 0.1 0.2; 0.1 0.1 0.1 0.0], [0.5, 0.3, 0.0, 0.2])

In [40]:
#output time, fill, time, fill with cresc/decr in abc notation
function abctune16patinst16ghost(
  vol,
  bass,
  pat,
  inst,
  fill,
  crdc)
    fp = ["!pp!","!p!","!mp!","!mf!","!f!","!ff!"]
    return("|:" * fp[vol] * abctime16patinst16ghost(bass, pat, inst) * "[1" * abcfill16(fill, 0) * ":|2" * abcfill16(fill, crdc)* "\n")
end


abctune16patinst16ghost (generic function with 1 method)

In [80]:
#put it all together, output abc notation
include("markov-drum-utilities.jl")  #loads insert, functionName, abptest
import Dates
function abcsolo16patinst16ghost(
  len::Int,
  basstrans::AugMatrix,
  filltrans::AugDblMatrix,
  dyntrans::AugDblMatrix,
  pattrans::AugMatrix,
  insttrans::AugMatrix,
  feel::String,
    )
    s = ""
    curdyn = markovrand(dyn.init)
    curpat = markovrand(pattrans.init)
    curinst = markovrand(insttrans.init)
    for i = 1:len
        vol = div(curdyn, 2) + 1
        bass = ghostbassgen(24, basstrans)
        pat = curpat + 1
        inst = curinst + 1
        fill = fill16gen(16, filltrans)
        crdc = 2 * mod(curdyn, 2) - 1
        s = s * abctune16patinst16ghost(vol, bass, pat, inst, fill, crdc)
        if i + 1 < len
            curdyn = markovrand(dyntrans.mat1[:, curdyn + 1])
        else
            curdyn = markovrand(dyntrans.mat2[:, curdyn + 1])
        end
        curpat = markovrand(pattrans.mat[:, curpat + 1])
        curinst = markovrand(insttrans.mat[:, curinst + 1])
    end
    s = insert(s, findlast(occursin("acbdefgABCDEFG"), s)-1, "!fermata!") #final fermata
    s = replace(s, "[]" => "z") #rest
    s = makeppheader("Computer-generated \\`Etude for Solo Drum Kit", String(@functionName), "$(Dates.now())", "4/4", feel, 80) * s * "|]\n"
    return(s)
end


abcsolo16patinst16ghost (generic function with 1 method)

In [81]:
function makeppheader(title::String, composer::String, date::String, meter::String, feel::String, tempo::Int)
    s = """%%abc-include percussions-JBH.abh
    
    I:linebreak \$
    
    #ifdef MIDI
    #define !>! \"[I:volinc 50]\"
    #define !Q! \"[I:vol 20]\" 
                    %%%ghost
    #define !U! \"[I:volinc -20]\" 
                    %%%unaccent
    #else
    #define !N! "\\\"^_\\\"" 
                    %%%tenuto
    #define !Q! !(.!!).! 
                    %%%ghost
    #define !U! !anti!  
                    %%%unaccent
    #endif
    
    X:1
    T:$title
    C:$composer
    O:$date
    M:$meter
    L:1/8
    Q:\"$feel\" 1/4=$tempo
    K:none clef=perc
    [V:1 clef=perc, stem=up]     % activate abc2xml.py map
    %%voicemap drummap  % activate abcm2ps/abc2svg map
    %%MIDI channel 10   % activate abc2midi map
    %%MIDI program 0
    %%flatbeams
    %%propagate-accidentals not
    %%pos ornament up
    %%ornament up
    %%MIDI fermatafixed
    """
    return(s)
end

makeppheader (generic function with 1 method)

In [ ]:
#test
print(makeppheader("Computer-generated \\`Etude for Solo Drum Kit", String(@functionName), "$(Dates.now())", "4/4", "Rock 16th feel", 80))

In [82]:
solo = abcsolo16patinst16ghost(4, rockBass, rockFill, dyn, rockPat, rockInst, "Rock 16th feel")
print(solo)

%%abc-include percussions-JBH.abh

I:linebreak $

#ifdef MIDI
#define !>! "[I:volinc 50]"
#define !Q! "[I:vol 20]" 
                %%%ghost
#define !U! "[I:volinc -20]" 
                %%%unaccent
#else
#define !N! "\"^_\"" 
                %%%tenuto
#define !Q! !(.!!).! 
                %%%ghost
#define !U! !anti!  
                %%%unaccent
#endif

X:1
T:Computer-generated \`Etude for Solo Drum Kit
C:abcsolo16patinst16ghost
O:2021-11-22T16:55:28.298
M:4/4
L:1/8
Q:"Rock 16th feel" 1/4=80
K:none clef=perc
[V:1 clef=perc, stem=up]     % activate abc2xml.py map
%%voicemap drummap  % activate abcm2ps/abc2svg map
%%MIDI channel 10   % activate abc2midi map
%%MIDI program 0
%%flatbeams
%%propagate-accidentals not
%%pos ornament up
%%ornament up
%%MIDI fermatafixed
|:!pp![^eF]/2[^e]/2[^e]/2[^e!Q!c]/2 [c^eF]/2[^eF]/2[^e]/2[^e]/2 [^eF]/2[^e]/2[^eF]/2[^e]/2 [c^eF]/2[^e]/2[^e]/2[^e!Q!c]/2 |
[^eF]/2[^e]/2[^eF]/2[^eF]/2 [c^e]/2[^e!Q!c]/2[^eF]/2[^eF]/2 [^e]/2[^e!Q!c]/2[^eF]/2[^e]/2 [c^e]/2[^e]/

In [83]:
#write a string to a file
open(f->print(f, solo), "rockghosttest.abp", "w")

In [84]:
run(`cat rockghosttest.abp`)

%%abc-include percussions-JBH.abh

I:linebreak $

#ifdef MIDI
#define !>! "[I:volinc 50]"
#define !Q! "[I:vol 20]" 
                %%%ghost
#define !U! "[I:volinc -20]" 
                %%%unaccent
#else
#define !N! "\"^_\"" 
                %%%tenuto
#define !Q! !(.!!).! 
                %%%ghost
#define !U! !anti!  
                %%%unaccent
#endif

X:1
T:Computer-generated \`Etude for Solo Drum Kit
C:abcsolo16patinst16ghost
O:2021-11-22T16:55:28.298
M:4/4
L:1/8
Q:"Rock 16th feel" 1/4=80
K:none clef=perc
[V:1 clef=perc, stem=up]     % activate abc2xml.py map
%%voicemap drummap  % activate abcm2ps/abc2svg map
%%MIDI channel 10   % activate abc2midi map
%%MIDI program 0
%%flatbeams
%%propagate-accidentals not
%%pos ornament up
%%ornament up
%%MIDI fermatafixed
|:!pp![^eF]/2[^e]/2[^e]/2[^e!Q!c]/2 [c^eF]/2[^eF]/2[^e]/2[^e]/2 [^eF]/2[^e]/2[^eF]/2[^e]/2 [c^eF]/2[^e]/2[^e]/2[^e!Q!c]/2 |
[^eF]/2[^e]/2[^eF]/2[^eF]/2 [c^e]/2[^e!Q!c]/2[^eF]/2[^eF]/2 [^e]/2[^e!Q!c]/2[^eF]/2[^e]/2 [c^e]/2[^e]/

Process(`cat rockghosttest.abp`, ProcessExited(0))

In [85]:
#Converts ABP file to score and MIDI and displays/plays it
abptest("rockghosttest.abp")

abcm2ps-8.14.12 (2021-07-14)
File rockghosttest-ps.abc
File percussions-JBH.abh
Output written on rockghosttest.xhtml (1 page, 1 title, 59126 bytes)
4.59 June 27 2021 abc2midi
writing MIDI file rockghosttest.mid
Warning in line-char 40-0 : Track 1 Bar 19 has 9/2 units instead of 4 in repeat


rockghosttest-ps.abc:29:47: warning: Line overfull (635pt of 628pt)


Process(`cmd /C start rockghosttest.xhtml`, ProcessExited(0))

In [107]:
NoGhostSolo = abcsolo16patinst16ghost(4, rockNoGhostB, rockFill, dyn, rockPat, rockInst, "Rock 16th feel")
print(NoGhostSolo)

%%abc-include percussions-JBH.abh

I:linebreak $

#ifdef MIDI
#define !>! "[I:volinc 50]"
#define !Q! "[I:vol 20]" 
                %%%ghost
#define !U! "[I:volinc -20]" 
                %%%unaccent
#else
#define !N! "\"^_\"" 
                %%%tenuto
#define !Q! !(.!!).! 
                %%%ghost
#define !U! !anti!  
                %%%unaccent
#endif

X:1
T:Computer-generated \`Etude for Solo Drum Kit
C:abcsolo16patinst16ghost
O:2021-11-22T16:55:28.298
M:4/4
L:1/8
Q:"Rock 16th feel" 1/4=80
K:none clef=perc
[V:1 clef=perc, stem=up]     % activate abc2xml.py map
%%voicemap drummap  % activate abcm2ps/abc2svg map
%%MIDI channel 10   % activate abc2midi map
%%MIDI program 0
%%flatbeams
%%propagate-accidentals not
%%pos ornament up
%%ornament up
%%MIDI fermatafixed
|:!pp![^eF]/2[^e]/2[^e]/2[^e!Q!c]/2 [c^eF]/2[^eF]/2[^e]/2[^e]/2 [^eF]/2[^e]/2[^eF]/2[^e]/2 [c^eF]/2[^e]/2[^e]/2[^e!Q!c]/2 |
[^eF]/2[^e]/2[^eF]/2[^eF]/2 [c^e]/2[^e!Q!c]/2[^eF]/2[^eF]/2 [^e]/2[^e!Q!c]/2[^eF]/2[^e]/2 [c^e]/2[^e]/

In [109]:
#write a string to a file
open(f->print(f, NoGhostSolo), "rocknoghosttest.abp", "w")
#Converts ABP file to score and MIDI and displays/plays it
abptest("rocknoghosttest.abp")

abcm2ps-8.14.12 (2021-07-14)
File rocknoghosttest-ps.abc
File percussions-JBH.abh
Output written on rocknoghosttest.xhtml (1 page, 1 title, 54401 bytes)
4.59 June 27 2021 abc2midi
Warning in line-char 24-5 : instruction !N! ignored
Warning in line-char 24-20 : instruction !N! ignored
Warning in line-char 24-33 : instruction !N! ignored
Warning in line-char 24-47 : instruction !N! ignored
Warning in line-char 24-60 : instruction !N! ignored
Warning in line-char 24-73 : instruction !N! ignored
Warning in line-char 24-89 : instruction !N! ignored
Warning in line-char 24-102 : instruction !N! ignored
Warning in line-char 25-0 : instruction !N! ignored
Warning in line-char 25-12 : instruction !N! ignored
Warning in line-char 25-25 : instruction !N! ignored
Warning in line-char 25-41 : instruction !N! ignored
Warning in line-char 25-54 : instruction !N! ignored
Warning in line-char 25-67 : instruction !N! ignored
Warning in line-char 25-80 : instruction !N! ignored
Warning in line-char 25-94

rocknoghosttest-ps.abc:29:47: warning: Line overfull (635pt of 628pt)


Process(`cmd /C start rocknoghosttest.xhtml`, ProcessExited(0))

In [68]:
#0 = rest, 1 = bass, 2 = LH
                #00    #01  #02   #10  #11   #12  #zeros
funkBassMatrix = [0.1e0 0.2  0     0.1  1//3 0   0 0 0 #00
                    0.2 0.2  0.2   0.2  1//3 0.1    0 0 0  #01
                    0.2 0.2  0.2   0.2  1//3 0.1    0 0 0  #02
                    0.1 0.2  0.2   0.1  0    0.3 0 0 0  #10
                    0.2 0    0.2   0.2  0    0.2 0 0 0  #11
                    0.2 0.2  0.2   0.2  0    0.3 0 0 0    #12
                    zeros(3,9)]                 #zeros

9×9 Matrix{Float64}:
 0.1  0.2  0.0  0.1  0.333333  0.0  0.0  0.0  0.0
 0.2  0.2  0.2  0.2  0.333333  0.1  0.0  0.0  0.0
 0.2  0.2  0.2  0.2  0.333333  0.1  0.0  0.0  0.0
 0.1  0.2  0.2  0.1  0.0       0.3  0.0  0.0  0.0
 0.2  0.0  0.2  0.2  0.0       0.2  0.0  0.0  0.0
 0.2  0.2  0.2  0.2  0.0       0.3  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0

In [69]:
sum(funkBassMatrix, dims=1)

1×9 Matrix{Float64}:
 1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0

In [7]:
funkBassInitial = [0.1, 0.2, 0.1, 0.2, 0.2, 0.2, 0, 0, 0]


9-element Vector{Float64}:
 0.1
 0.2
 0.1
 0.2
 0.2
 0.2
 0.0
 0.0
 0.0

In [8]:
sum(funkBassInitial, dims=1)

1-element Vector{Float64}:
 1.0

In [46]:
funkBass = AugMatrix(funkBassMatrix, funkBassInitial)

AugMatrix([0.1 0.2 … 0.0 0.0; 0.2 0.2 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.1, 0.2, 0.1, 0.2, 0.2, 0.2, 0.0, 0.0, 0.0])

In [110]:
funkNoGhostBMatrix = [0.1e0 1//3 0.25e0 0.5e0; 0.3e0 1//3 0.25e0 0.5e0; 0.3e0 1//3 0.25e0 0; 0.3e0 0 0.25e0 0]

4×4 Matrix{Float64}:
 0.1  0.333333  0.25  0.5
 0.3  0.333333  0.25  0.5
 0.3  0.333333  0.25  0.0
 0.3  0.0       0.25  0.0

In [111]:
funkNoGhostBInitial = [0.1e0, 0.1e0, 0, 0.4e0, 0.4e0, 0, 0, 0, 0 ]

9-element Vector{Float64}:
 0.1
 0.1
 0.0
 0.4
 0.4
 0.0
 0.0
 0.0
 0.0

In [112]:
funkNoGhostB = AugMatrix([funkNoGhostBMatrix[1:2, 1:2] zeros(2,1) funkNoGhostBMatrix[1:2, 3:4] zeros(2, 4)
    zeros(1, 9)
    funkNoGhostBMatrix[3:4, 1:2] zeros(2,1) funkNoGhostBMatrix[3:4, 3:4] zeros(2, 4)
    zeros(4, 9)
    ],
    funkNoGhostBInitial)

AugMatrix([0.1 0.3333333333333333 … 0.0 0.0; 0.3 0.3333333333333333 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.1, 0.1, 0.0, 0.4, 0.4, 0.0, 0.0, 0.0, 0.0])

In [47]:
funkFillLeftToRight = [0.2e0 0.1e0 0.1e0 0 0.3e0 0.2e0; 0.4e0 0.2e0 0 0.3e0 0.3e0 0.2e0; 0.1e0 0.4e0 0.2e0 0.1e0 0.1e0 0.2e0; 0.1e0 0.1e0 0.4e0 0.3e0 0.1e0 0.2e0; 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.1e0; 0.1e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0]

6×6 Matrix{Float64}:
 0.2  0.1  0.1  0.0  0.3  0.2
 0.4  0.2  0.0  0.3  0.3  0.2
 0.1  0.4  0.2  0.1  0.1  0.2
 0.1  0.1  0.4  0.3  0.1  0.2
 0.1  0.1  0.1  0.2  0.1  0.1
 0.1  0.1  0.2  0.1  0.1  0.1

In [48]:
funkFillRightToLeft = [0.4e0 0.2e0 0.1e0 0.1e0 0.3e0 0.2e0; 0.2e0 0.4e0 0.2e0 0.2e0 0.2e0 0.2e0; 0.1e0 0 0.4e0 0.1e0 0.1e0 0.2e0; 0 0.1e0 0.1e0 0.4e0 0.1e0 0.2e0; 0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0; 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0]


6×6 Matrix{Float64}:
 0.4  0.2  0.1  0.1  0.3  0.2
 0.2  0.4  0.2  0.2  0.2  0.2
 0.1  0.0  0.4  0.1  0.1  0.2
 0.0  0.1  0.1  0.4  0.1  0.2
 0.2  0.1  0.1  0.1  0.2  0.1
 0.1  0.2  0.1  0.1  0.1  0.1

In [49]:
funkFillInitial = [0.3e0,0.3e0,0.1e0,0.1e0,0.1e0,0.1e0]

6-element Vector{Float64}:
 0.3
 0.3
 0.1
 0.1
 0.1
 0.1

In [50]:
funkFill = AugDblMatrix(funkFillLeftToRight, funkFillRightToLeft, funkFillInitial)

AugDblMatrix([0.2 0.1 … 0.3 0.2; 0.4 0.2 … 0.3 0.2; … ; 0.1 0.1 … 0.1 0.1; 0.1 0.1 … 0.1 0.1], [0.4 0.2 … 0.3 0.2; 0.2 0.4 … 0.2 0.2; … ; 0.2 0.1 … 0.2 0.1; 0.1 0.2 … 0.1 0.1], [0.3, 0.3, 0.1, 0.1, 0.1, 0.1])

In [52]:
funkPatMatrix = [
    #8  Q   b   +
    0   0.2 0.2 0.2 #8 
    0.2 0   0.4 0.4   #Q
    0.2 0.2 0   0.4 #b
    0.6 0.6 0.4 0 #+
    ]

4×4 Matrix{Float64}:
 0.0  0.2  0.2  0.2
 0.2  0.0  0.4  0.4
 0.2  0.2  0.0  0.4
 0.6  0.6  0.4  0.0

In [53]:
sum(funkPatMatrix, dims=1)

1×4 Matrix{Float64}:
 1.0  1.0  1.0  1.0

In [54]:
funkPatInitial = [0; 0.2; 0.4; 0.4]

4-element Vector{Float64}:
 0.0
 0.2
 0.4
 0.4

In [55]:
funkPat = AugMatrix(funkPatMatrix, funkPatInitial)

AugMatrix([0.0 0.2 0.2 0.2; 0.2 0.0 0.4 0.4; 0.2 0.2 0.0 0.4; 0.6 0.6 0.4 0.0], [0.0, 0.2, 0.4, 0.4])

In [56]:
funkInstMatrix = [
   #HH  RC  RB  ST 
    0.1 0.4 0.4   0.4   #HH
    0.6 0.1 0.4   0.4   #RC
    0.2 0.4 0.1   0.2   #RB
    0.1 0.1 0.1   0     #ST
]

4×4 Matrix{Float64}:
 0.1  0.4  0.4  0.4
 0.6  0.1  0.4  0.4
 0.2  0.4  0.1  0.2
 0.1  0.1  0.1  0.0

In [57]:
funkInstInitial = [0.4; 0.4; 0.2; 0]

4-element Vector{Float64}:
 0.4
 0.4
 0.2
 0.0

In [58]:
funkInst = AugMatrix(funkInstMatrix, funkInstInitial)

AugMatrix([0.1 0.4 0.4 0.4; 0.6 0.1 0.4 0.4; 0.2 0.4 0.1 0.2; 0.1 0.1 0.1 0.0], [0.4, 0.4, 0.2, 0.0])

In [86]:
funksolo = abcsolo16patinst16ghost(4, funkBass, funkFill, dyn, funkPat, funkInst, "Funk 16th feel")
print(funksolo)

%%abc-include percussions-JBH.abh

I:linebreak $

#ifdef MIDI
#define !>! "[I:volinc 50]"
#define !Q! "[I:vol 20]" 
                %%%ghost
#define !U! "[I:volinc -20]" 
                %%%unaccent
#else
#define !N! "\"^_\"" 
                %%%tenuto
#define !Q! !(.!!).! 
                %%%ghost
#define !U! !anti!  
                %%%unaccent
#endif

X:1
T:Computer-generated \`Etude for Solo Drum Kit
C:abcsolo16patinst16ghost
O:2021-11-22T16:58:58.629
M:4/4
L:1/8
Q:"Funk 16th feel" 1/4=80
K:none clef=perc
[V:1 clef=perc, stem=up]     % activate abc2xml.py map
%%voicemap drummap  % activate abcm2ps/abc2svg map
%%MIDI channel 10   % activate abc2midi map
%%MIDI program 0
%%flatbeams
%%propagate-accidentals not
%%pos ornament up
%%ornament up
%%MIDI fermatafixed
|:!p![^g]/2[^gF]/2[^g^DF]/2[^g!Q!c]/2 [c^g]/2[^gF]/2[^g^DF]/2[^g]/2 [^gF]/2[^g]/2[^g^DF]/2[^g!Q!c]/2 [c^gF]/2[^g]/2[^g^D]/2[^gF]/2 |
[^g]/2[^g]/2[^g^D]/2[^g!Q!c]/2 [c^g]/2[^gF]/2[^g^D]/2[^g]/2 [^g]/2[^g]/2[^g^D]/2[^g!Q!c]/2 [c

In [87]:
#write a string to a file
open(f->print(f, funksolo), "funkghosttest.abp", "w")

In [88]:
run(`cat funkghosttest.abp`)

%%abc-include percussions-JBH.abh

I:linebreak $

#ifdef MIDI
#define !>! "[I:volinc 50]"
#define !Q! "[I:vol 20]" 
                %%%ghost
#define !U! "[I:volinc -20]" 
                %%%unaccent
#else
#define !N! "\"^_\"" 
                %%%tenuto
#define !Q! !(.!!).! 
                %%%ghost
#define !U! !anti!  
                %%%unaccent
#endif

X:1
T:Computer-generated \`Etude for Solo Drum Kit
C:abcsolo16patinst16ghost
O:2021-11-22T16:58:58.629
M:4/4
L:1/8
Q:"Funk 16th feel" 1/4=80
K:none clef=perc
[V:1 clef=perc, stem=up]     % activate abc2xml.py map
%%voicemap drummap  % activate abcm2ps/abc2svg map
%%MIDI channel 10   % activate abc2midi map
%%MIDI program 0
%%flatbeams
%%propagate-accidentals not
%%pos ornament up
%%ornament up
%%MIDI fermatafixed
|:!p![^g]/2[^gF]/2[^g^DF]/2[^g!Q!c]/2 [c^g]/2[^gF]/2[^g^DF]/2[^g]/2 [^gF]/2[^g]/2[^g^DF]/2[^g!Q!c]/2 [c^gF]/2[^g]/2[^g^D]/2[^gF]/2 |
[^g]/2[^g]/2[^g^D]/2[^g!Q!c]/2 [c^g]/2[^gF]/2[^g^D]/2[^g]/2 [^g]/2[^g]/2[^g^D]/2[^g!Q!c]/2 [c

Process(`cat funkghosttest.abp`, ProcessExited(0))

In [89]:
#Converts ABP file to score and MIDI and displays/plays it
abptest("funkghosttest.abp")

abcm2ps-8.14.12 (2021-07-14)
File funkghosttest-ps.abc
File percussions-JBH.abh
Output written on funkghosttest.xhtml (1 page, 1 title, 62007 bytes)
4.59 June 27 2021 abc2midi
writing MIDI file funkghosttest.mid
Warning in line-char 40-0 : Track 1 Bar 19 has 9/2 units instead of 4 in repeat


funkghosttest-ps.abc:29:47: warning: Line overfull (635pt of 628pt)


Process(`cmd /C start funkghosttest.xhtml`, ProcessExited(0))

In [113]:
funkNoGhostsolo = abcsolo16patinst16ghost(4, funkNoGhostB, funkFill, dyn, funkPat, funkInst, "Funk 16th feel")
print(funkNoGhostsolo)

%%abc-include percussions-JBH.abh

I:linebreak $

#ifdef MIDI
#define !>! "[I:volinc 50]"
#define !Q! "[I:vol 20]" 
                %%%ghost
#define !U! "[I:volinc -20]" 
                %%%unaccent
#else
#define !N! "\"^_\"" 
                %%%tenuto
#define !Q! !(.!!).! 
                %%%ghost
#define !U! !anti!  
                %%%unaccent
#endif

X:1
T:Computer-generated \`Etude for Solo Drum Kit
C:abcsolo16patinst16ghost
O:2021-11-22T19:04:01.226
M:4/4
L:1/8
Q:"Funk 16th feel" 1/4=80
K:none clef=perc
[V:1 clef=perc, stem=up]     % activate abc2xml.py map
%%voicemap drummap  % activate abcm2ps/abc2svg map
%%MIDI channel 10   % activate abc2midi map
%%MIDI program 0
%%flatbeams
%%propagate-accidentals not
%%pos ornament up
%%ornament up
%%MIDI fermatafixed
|:!f!!>![^eF]/2!U![^e]/2!>![^eF]/2!U![^e]/2 !>![c^e]/2!U![^e]/2!>![^eF]/2!U![^e]/2 !>![^e]/2!U![^eF]/2!>![^eF]/2!U![^e]/2 !>![c^eF]/2!U![^e]/2!>![^eF]/2!U![^e]/2 |
!>![^e]/2!U![^e]/2!>![^eF]/2!U![^e]/2 !>![c^e]/2!U![^eF]/2!>![

In [117]:
#write a string to a file
open(f->print(f, funkNoGhostsolo), "funknoghosttest.abp", "w")

In [118]:
#Converts ABP file to score and MIDI and displays/plays it
abptest("funknoghosttest.abp")

abcm2ps-8.14.12 (2021-07-14)
File funknoghosttest-ps.abc
File percussions-JBH.abh
Output written on funknoghosttest.xhtml (1 page, 1 title, 58857 bytes)


funknoghosttest-ps.abc:29:47: warning: Line overfull (635pt of 628pt)


4.59 June 27 2021 abc2midi
writing MIDI file funknoghosttest.mid
Warning in line-char 40-0 : Track 1 Bar 19 has 9/2 units instead of 4 in repeat


Process(`cmd /C start funknoghosttest.xhtml`, ProcessExited(0))